# Necessary Import 

In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import re
import string
import json
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


import nltk
os.makedirs(r'C:\\nltk_data', exist_ok=True)
nltk.data.path.append(r'C:\\nltk_data')  
nltk.download('punkt_tab', download_dir=r'C:\\nltk_data')
nltk.download('stopwords', download_dir=r'C:\\nltk_data')
nltk.download('wordnet', download_dir=r'C:\\nltk_data')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn.functional as F
from tqdm import tqdm

[nltk_data] Downloading package punkt_tab to C:\\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to C:\\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [33]:
def read_database_result_csv(file_path):
    try:
        df = pd.read_csv(file_path, sep=';', encoding='utf-8')
    except UnicodeDecodeError as e1:
        try:
            df = pd.read_csv(file_path, sep=';', encoding='latin1')
        except UnicodeDecodeError as e2:
            try:
                df = pd.read_csv(file_path, sep=';', encoding='cp1252')
            except Exception as e3:
                print(f"Gagal membaca file {file_path}: {e1} | {e2} | {e3}")
                return pd.DataFrame()  
    return df

file_path = 'data/main_data_19.csv'
database_result_df = read_database_result_csv(file_path)

print(database_result_df)

                  Dataset                      ID BIDANG SATKER (AKRONIM)  \
0       Database Internal    DATABASE INTERNAL_23     MS             DPSI   
1       Database Internal    DATABASE INTERNAL_24     MS             DPSI   
2       Database Internal    DATABASE INTERNAL_25     MS             DPSI   
3       Database Internal    DATABASE INTERNAL_26     MS             DPSI   
4       Database Internal    DATABASE INTERNAL_28     MS             DPSI   
...                   ...                     ...    ...              ...   
8242  DATABASE KOJK NO HP  EKSTERNAL_KOJK_HP_5221     KS       KR3 - KOSG   
8243  DATABASE KOJK NO HP  EKSTERNAL_KOJK_HP_5222     KS       KR3 - KOSG   
8244  DATABASE KOJK NO HP  EKSTERNAL_KOJK_HP_5230     KS       KR3 - KOSG   
8245  DATABASE KOJK NO HP  EKSTERNAL_KOJK_HP_5232     KS       KR3 - KOSG   
8246  DATABASE KOJK NO HP  EKSTERNAL_KOJK_HP_5235     KS       KR3 - KOSG   

      JENIS SURVEI                           TIPE QUESTION  \
0         INT

In [34]:
columns_order = [
    'ID',
    'BIDANG',
    'SATKER (AKRONIM)',
    'JENIS SURVEI',
    'TIPE QUESTION',
    'INSTITUSI / PERSEORANGAN/ASAL SATKER',
    'RESPOND',
    'LINK SURVEYMONKEY',
    'TOKEN',
    'NAMA PIC/RESPONDEN',
    'JABATAN/PROFESI/LVEL DI OJK',
    'KONTAK',
    'FUNGSI YANG DINILAI',
    'DIRECT / INDIRECT',
    'JENIS STAKEHOLDERS',
    'RELASI RESPONDEN DENGAN SATKER',
    'POWER',
    'INTEREST',
    'KATEGORI',
    'Dataset',
    'RESOURCE PERCEPTION',
    'PERFORMANCE DELIVERY',
    'OPEN QUESTION 1',
    'OPEN QUESTION 2'
]

In [35]:
all_data_idi_df = database_result_df[columns_order]
print(all_data_idi_df)

                          ID BIDANG SATKER (AKRONIM)  JENIS SURVEI  \
0       DATABASE INTERNAL_23     MS             DPSI      INTERNAL   
1       DATABASE INTERNAL_24     MS             DPSI      INTERNAL   
2       DATABASE INTERNAL_25     MS             DPSI      INTERNAL   
3       DATABASE INTERNAL_26     MS             DPSI      INTERNAL   
4       DATABASE INTERNAL_28     MS             DPSI      INTERNAL   
...                      ...    ...              ...           ...   
8242  EKSTERNAL_KOJK_HP_5221     KS       KR3 - KOSG  NON INTERNAL   
8243  EKSTERNAL_KOJK_HP_5222     KS       KR3 - KOSG  NON INTERNAL   
8244  EKSTERNAL_KOJK_HP_5230     KS       KR3 - KOSG  NON INTERNAL   
8245  EKSTERNAL_KOJK_HP_5232     KS       KR3 - KOSG  NON INTERNAL   
8246  EKSTERNAL_KOJK_HP_5235     KS       KR3 - KOSG  NON INTERNAL   

                               TIPE QUESTION  \
0                                DIRECT DPSI   
1                                DIRECT DPSI   
2              

In [36]:
print(all_data_idi_df.isnull().sum())

ID                                         0
BIDANG                                     0
SATKER (AKRONIM)                           0
JENIS SURVEI                               0
TIPE QUESTION                              0
INSTITUSI / PERSEORANGAN/ASAL SATKER       3
RESPOND                                    0
LINK SURVEYMONKEY                          0
TOKEN                                      0
NAMA PIC/RESPONDEN                        14
JABATAN/PROFESI/LVEL DI OJK              239
KONTAK                                   185
FUNGSI YANG DINILAI                        0
DIRECT / INDIRECT                          0
JENIS STAKEHOLDERS                         3
RELASI RESPONDEN DENGAN SATKER           941
POWER                                      0
INTEREST                                   0
KATEGORI                                   0
Dataset                                    0
RESOURCE PERCEPTION                     3793
PERFORMANCE DELIVERY                    3793
OPEN QUEST

In [37]:
all_data_idi_df.fillna("-", inplace=True)
print(all_data_idi_df.isnull().sum())

C:\Users\hp\AppData\Local\Temp\ipykernel_11744\1738321216.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_idi_df.fillna("-", inplace=True)


ID                                      0
BIDANG                                  0
SATKER (AKRONIM)                        0
JENIS SURVEI                            0
TIPE QUESTION                           0
INSTITUSI / PERSEORANGAN/ASAL SATKER    0
RESPOND                                 0
LINK SURVEYMONKEY                       0
TOKEN                                   0
NAMA PIC/RESPONDEN                      0
JABATAN/PROFESI/LVEL DI OJK             0
KONTAK                                  0
FUNGSI YANG DINILAI                     0
DIRECT / INDIRECT                       0
JENIS STAKEHOLDERS                      0
RELASI RESPONDEN DENGAN SATKER          0
POWER                                   0
INTEREST                                0
KATEGORI                                0
Dataset                                 0
RESOURCE PERCEPTION                     0
PERFORMANCE DELIVERY                    0
OPEN QUESTION 1                         0
OPEN QUESTION 2                   

In [38]:
all_data_idi_df.describe()

,ID,BIDANG,SATKER (AKRONIM),JENIS SURVEI,TIPE QUESTION,INSTITUSI / PERSEORANGAN/ASAL SATKER,RESPOND,LINK SURVEYMONKEY,TOKEN,NAMA PIC/RESPONDEN,...,JENIS STAKEHOLDERS,RELASI RESPONDEN DENGAN SATKER,POWER,INTEREST,KATEGORI,Dataset,RESOURCE PERCEPTION,PERFORMANCE DELIVERY,OPEN QUESTION 1,OPEN QUESTION 2
count,8247,8247,8247,8247,8247,8247,8247,8247,8247,8247,...,8247,8247,8247,8247,8247,8247,8247,8247,8247,8247
unique,8247,9,56,2,73,1437,1,25,237,4040,...,183,238,3,3,4,7,26,90,3269,4538
top,EKSTERNAL_KOJK_HP_5235,MS,DPSI,INTERNAL,DIRECT DPSI,DPW1,SUDAH DI ISI,"Internal 1 (Direct DPSI & Indirect DOSB, 253_2...",Token 0252,-,...,PEGAWAI OJK,PENERIMA HELPDESK & LAYANAN DPSI LAINNYA,HIGH,HIGH,PLAYER,Database Internal,-,-,-,-
freq,1,5362,2474,5940,2458,205,8247,3334,3334,14,...,5919,2458,5364,7722,5117,4106,3793,3793,2446,2822


In [39]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+|\@\w+|\#|\d+|[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = word_tokenize(text)
    stop_words = set(stopwords.words('indonesian'))
    words = [word for word in words if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    cleaned_text = ' '.join(words)
    
    return cleaned_text

In [40]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
import pandas as pd
from tqdm import tqdm

# Ganti model_name dengan model IndoBERT
model_name = "indobenchmark/indobert-base-p2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=6  # Sesuaikan jumlah label
)

label_map = {
    0: "sangat tidak setuju",
    1: "tidak setuju",
    2: "kurang setuju",
    3: "cukup setuju",
    4: "setuju",
    5: "sangat setuju"
}

# Fungsi untuk memproses teks
def preprocess_text(text):
    return text.strip().lower() if isinstance(text, str) else ""  # Periksa apakah teks valid

# Fungsi untuk prediksi sentimen
def predict_sentiment(texts):
    model.eval()
    results = []
    
    with torch.no_grad():
        for text in tqdm(texts):
            text = preprocess_text(text)
            
            if not text:  # Lewati teks kosong
                results.append({
                    'text': text,
                    'sentiment': "unknown",
                    'confidence': 0.0
                })
                continue
            
            # Tokenisasi teks
            encoded = tokenizer(
                text,
                truncation=True,
                padding=True,
                max_length=512,
                return_tensors="pt"
            )
            
            # Model memprediksi
            outputs = model(encoded["input_ids"], attention_mask=encoded["attention_mask"])
            predictions = F.softmax(outputs.logits, dim=1)
            predicted_label = torch.argmax(predictions, dim=1).item()
            
            # Ambil confidence score
            confidence = predictions[0][predicted_label].item()
            
            # Mapping hasil prediksi ke label
            sentiment = label_map.get(predicted_label, "unknown")
            
            # Menambahkan hasil ke dalam list
            results.append({
                'text': text,
                'sentiment': sentiment,
                'confidence': confidence
            })
    
    return pd.DataFrame(results)

columns_to_process = ['OPEN QUESTION 1','OPEN QUESTION 2']

all_data_idi_df['Text'] = all_data_idi_df[columns_to_process].fillna("").apply(lambda row: " ".join(row), axis=1)

results = predict_sentiment(all_data_idi_df['Text'].tolist())

all_data_idi_df.loc[:, 'Label'] = results['sentiment']

# Menampilkan hasil
print("\nSample of labeled data:")
print(all_data_idi_df[['OPEN QUESTION 1','OPEN QUESTION 2','Label']].head())


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\hp\AppData\Local\Temp\ipykernel_11744\818057034.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_idi_df['Text'] = all_data_idi_df[columns_to_process].fillna("").apply(lambda row: " ".join(row), axis=1)
100%|██████████| 8247/8247 [08:58<00:00, 15.31it/s]


Sample of labeled data:
                                     OPEN QUESTION 1  \
0  Sdh sangat bagus hanya banyak terkendala krn k...   
1                                        sangat baik   
2  Cukup puas. Perlu di tingkatkan lagi terkait b...   
3                            sudah baik, pertahankan   
4                     Sejauh ini baik dan responsif.   

                                     OPEN QUESTION 2          Label  
0  fungsi pengelolaan informasi harus imbang anta...         setuju  
1                                                  -  kurang setuju  
2  Agar dibuat single sign on, pusat database ata...         setuju  
3  sudah cukup baik pertahankan kinerja yang suda...  sangat setuju  
4  Agar terdapat no wa helpdesk per masing-masing...         setuju  



C:\Users\hp\AppData\Local\Temp\ipykernel_11744\818057034.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_idi_df.loc[:, 'Label'] = results['sentiment']


In [41]:
all_data_idi_df.to_csv('data/hasil/main_data_19_OQ2.csv', index=False, sep=';')
print(all_data_idi_df)

                          ID BIDANG SATKER (AKRONIM)  JENIS SURVEI  \
0       DATABASE INTERNAL_23     MS             DPSI      INTERNAL   
1       DATABASE INTERNAL_24     MS             DPSI      INTERNAL   
2       DATABASE INTERNAL_25     MS             DPSI      INTERNAL   
3       DATABASE INTERNAL_26     MS             DPSI      INTERNAL   
4       DATABASE INTERNAL_28     MS             DPSI      INTERNAL   
...                      ...    ...              ...           ...   
8242  EKSTERNAL_KOJK_HP_5221     KS       KR3 - KOSG  NON INTERNAL   
8243  EKSTERNAL_KOJK_HP_5222     KS       KR3 - KOSG  NON INTERNAL   
8244  EKSTERNAL_KOJK_HP_5230     KS       KR3 - KOSG  NON INTERNAL   
8245  EKSTERNAL_KOJK_HP_5232     KS       KR3 - KOSG  NON INTERNAL   
8246  EKSTERNAL_KOJK_HP_5235     KS       KR3 - KOSG  NON INTERNAL   

                               TIPE QUESTION  \
0                                DIRECT DPSI   
1                                DIRECT DPSI   
2              

In [42]:
# cek lagi
label_counts = all_data_idi_df['Label'].value_counts()

label_summary = pd.DataFrame(label_counts).reset_index()
label_summary.columns = ['Label', 'Count']
print(label_summary)

                 Label  Count
0               setuju   3824
1        kurang setuju   3054
2        sangat setuju    534
3  sangat tidak setuju    416
4         cukup setuju    403
5         tidak setuju     16
